# Classification of Mushrooms using Machine Learning | Machine learning 

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import pickle
import datetime

### Running on GPUs

In [3]:


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9630609941117404267
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6957301760
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4580495808528678306
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:26:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
Default GPU Device: /device:GPU:0
1 Physical GPUs, 1 Logical GPU


In [4]:
X  = pickle.load(open("./X.pickle","rb"))
y  = pickle.load(open("./y.pickle","rb"))
y = np.array(y)
X = X/255.0
conv_size = 16
number_of_classes =len(set(y))
#print(number_of_classes)
#print(X,y)



### Model

In [5]:
%pip install -U scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


#Converty true labels y to one-hot econded vectors

y = to_categorical(y,num_classes = number_of_classes)

X = X.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


NAME = f"Mushrooms-cnn-264x264x128x128-{number_of_classes}-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(NAME)
tensorboard = TensorBoard(log_dir = f"logs/{NAME}")

model = Sequential(
    [
        Conv2D(264,kernel_size = (3,3),activation = 'relu', input_shape=(X.shape[1],X.shape[2],X.shape[3])),
        MaxPooling2D(pool_size =(2,2)),

        Dense(128,activation = 'relu'),
        Dropout(0.5),
        Dense(128,activation = 'relu'),
        
        #Dense(64,activation = 'relu'),
        #Dense(32,activation = 'relu'),
        #Dense(16,activation = 'relu'),
        Flatten(),

        Dense(number_of_classes,activation='softmax')
    ]
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Mushrooms-cnn-264x264x128x128-9-20230404-222759
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 264)     2640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 264)      0         
 )                                                               
                                                                 
 dense (Dense)               (None, 99, 99, 128)       33920     
                                                                 
 dropout (Dropout)           (None, 99, 99, 128)       0         
                                                                 
 dense_1 (Dense)             (None, 99, 99, 128)       16512     
                                                                 
 flatten (Flatten)           (None, 1254528)           0         
        

In [7]:

model.fit(X_train,y_train,epochs=10,batch_size=8,validation_split=0.2,callbacks=[tensorboard],use_multiprocessing=True)

Epoch 1/10
450/450 [==============================] - 42s 71ms/step - loss: 2.1216 - accuracy: 0.2296 - val_loss: 2.0364 - val_accuracy: 0.2400
Epoch 2/10
450/450 [==============================] - 31s 68ms/step - loss: 2.0532 - accuracy: 0.2273 - val_loss: 2.0417 - val_accuracy: 0.2400
Epoch 3/10
450/450 [==============================] - 31s 68ms/step - loss: 2.0558 - accuracy: 0.2321 - val_loss: 2.0956 - val_accuracy: 0.2400
Epoch 4/10
450/450 [==============================] - 31s 68ms/step - loss: 2.0559 - accuracy: 0.2332 - val_loss: 2.0528 - val_accuracy: 0.2400
Epoch 5/10
450/450 [==============================] - 31s 68ms/step - loss: 2.0576 - accuracy: 0.2301 - val_loss: 2.0372 - val_accuracy: 0.2400
Epoch 6/10
450/450 [==============================] - 31s 68ms/step - loss: 1.9331 - accuracy: 0.3196 - val_loss: 2.0565 - val_accuracy: 0.2522
Epoch 7/10
450/450 [==============================] - 31s 68ms/step - loss: 1.0528 - accuracy: 0.6520 - val_loss: 3.2115 - val_accuracy:

KeyboardInterrupt: 

In [8]:
model.evaluate(X_test,y_test, verbose = 1)

70/70 [==============================] - 9s 85ms/step - loss: 6.6769 - accuracy: 0.2180


[6.676882743835449, 0.21796028316020966]